In [92]:
using BenchmarkTools: @btime
import LinearAlgebra as la

In [8]:
struct HO_Funcs
    ω::Float64
    hermites::Vector{Float64}
    hos::Vector{Float64}
    
    function HO_Funcs(l, ω)
        hermites = zeros(l)
        hermites[1] = 1.0
        
        hos = zeros(l)
        return new(ω, hermites, hos)
    end
end

In [17]:
function fast_ho(x, ho)
    ω, hermites, hos = ho.ω, ho.hermites, ho.hos
    
    x = √ω * x
    hermites[2] = 2x

    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    hos[1] = ho_fac * hermites[1]
    ho_fac *= 1 / √2
    hos[2] = ho_fac * hermites[2]

    for n in 3:length(hos)
        hermites[n] = 2x * hermites[n-1] - 2(n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt( 2(n - 1) )
        hos[n] = ho_fac * hermites[n]
    end
    
    return hos
end

fast_ho (generic function with 1 method)

In [103]:
fast_ho(8.1, ho)

50-element Vector{Float64}:
  0.00014569341203507453
  0.0008344684769560763
  0.003276576532047902
  0.010153679160712206
  0.026240329809357737
  0.05813143235875152
  0.11197286813391592
  0.18858141465722794
  0.27713633738800386
  0.3513091925835325
  0.37338188638435676
  0.30984264138949413
  0.15480961150369804
  ⋮
  0.19178930666533067
  0.0307608875536246
 -0.16151947509417727
 -0.1748618602721361
  0.005045196103816062
  0.17722332560422738
  0.14803823206759484
 -0.04884585615130985
 -0.18766979286386054
 -0.10846552987210341
  0.09603585402138545
  0.18593183195993027

In [96]:
l = 50
ω = 0.25
ho = HO_Funcs(l, ω);

In [97]:
n = 4
positions = rand(4)
HF = rand(l, n);

In [98]:
slater = zeros(n, n);

In [99]:
function setupSlater!(slater, positions, ho, HF)
    n, l = size(slater)[1], length(ho.hos)
    for (row, x) in enumerate(positions)
        hos = fast_ho(x, ho) # all l basis functions evaluated on x

        for col in 1:n
            ϕ_col_x = 0.0
            for i in 1:l
                ϕ_col_x += HF[i, col] * hos[i]
            end
            slater[row, col] = ϕ_col_x
        end
    end
    return slater
end

setupSlater! (generic function with 2 methods)

In [100]:
@btime setupSlater!(slater, positions, ho, HF)

  1.450 μs (0 allocations: 0 bytes)


4×4 Matrix{Float64}:
  0.478584   -0.0355856  0.484533  0.0207133
 -0.0237272  -0.393473   0.613893  0.572105
  0.0344489  -0.393559   0.604177  0.518043
 -0.123132   -0.346754   0.637647  0.638696

In [101]:
function updateSlater!(slater, i, x, ho, HF)
    n, l = size(slater)[1], length(ho.hos)
    hos = fast_ho(x, ho) # all l basis functions evaluated on x
    
    for col in 1:n
        ϕ_col_x = 0.0
        for j in 1:l
            ϕ_col_x += HF[j, col] * hos[j]
        end
        slater[i, col] = ϕ_col_x
    end
    return slater
end

updateSlater! (generic function with 2 methods)

In [102]:
i = 1
x = 0.314159279
@btime updateSlater!(slater, i, x, ho, HF)

  359.716 ns (0 allocations: 0 bytes)


4×4 Matrix{Float64}:
  0.385062   -0.185127  0.550859  0.155598
 -0.0237272  -0.393473  0.613893  0.572105
  0.0344489  -0.393559  0.604177  0.518043
 -0.123132   -0.346754  0.637647  0.638696

In [95]:
la.det(slater)

-1.701265821540461e-5